# how to run deits solver
1. clone the deits repository (https://github.com/rdeits/cryptics) into './deits/; Note that the julia solver is better but for our research we used python version.
2. checkout commit 402579 (in deits repo)
3. apply patch 0001-Set-up... (in this directory)
4. Use this notebook to set up the deits input clue file
5. run validate_cryptics.py in the deits directory.
    - the patch will create this file. (see bottom of validate_cryptics.py for command line arguments)
    - you will need to specify an output file. Use the abs path of '../deits/clues/'
    - you will need to specify input clue file. that should be the one generated in this nb ('../deits/clues/*')
6. Use this notebook to run eval (this file)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import config
k_json_folder = config.DataDirs.Guardian.json_folder
k_deits_clue_folder = config.DataDirs.Deits.k_deits_clues
k_deits_output_folder = config.DataDirs.Deits.k_deits_outputs

In [ ]:
import sys
sys.path.append('../decrypt')
from decrypt.scrape_parse import (
    load_guardian_splits,
    load_guardian_splits_disjoint_hash
)

In [2]:
# from data_util.scrape_parse.guardian_scrape import load_guardian_splits, load_guardian_splits_disjoint_hash
from decrypt.common.validation_tools import (
    load_deits,
    all_aggregate,
)
from decrypt.common.puzzle_clue import GuardianClue

initializing a csv writer


INFO:data_util.baselines.validation_tools:Opened clue labels file:


In [3]:
## produce deits

# produce test set for cdeits
def make_deits_format(gc: GuardianClue):
    len_str = "(" + ",".join(map(str, gc.lengths)) + ")"
    clue_str = gc.clue + " " + len_str
    final = clue_str + " | " + gc.soln_with_spaces
    return final

def make_deits(fname, clueset):
    with open(fname, "w") as f:
        for c in clueset:
            f.write(make_deits_format(c) + "\n")


In [12]:
def prep_deits(val_or_test: str,
               naive_or_disj: str):
    assert val_or_test in ["val", "test"]
    assert naive_or_disj in ["naive", "disj"]
    if naive_or_disj == "naive":
        load_fn = load_guardian_splits
    else:
        load_fn = load_guardian_splits_disjoint_hash

    _, _, (train_local, val, test) = load_fn(k_json_folder)
    if val_or_test == "val":
        val_local = val
    else:
        val_local = test

    append_name = f'guardian_{naive_or_disj}_{val_or_test}'
    fname = f'{k_deits_clue_folder}{append_name}.txt'
    output_folder = f'{k_deits_output_folder}{append_name}/'        # outputs will be like output/{append_name}/*-1.json
    return val_local, fname, output_folder

def make_deits_clues(val_or_test,
                     naive_or_disj):
    val_local, fname, _ = prep_deits(val_or_test, naive_or_disj)
    make_deits(fname, val_local)

def eval_deits_clues(val_or_test,
                     naive_or_disj):
    val_local, fname, output_folder = prep_deits(val_or_test, naive_or_disj)
    deits_outputs_glob = output_folder + "*-1.json"
    model_outputs = load_deits(val_local, deits_outputs_glob)
    all_aggregate(model_outputs)


In [ ]:
# e.g. for disjoint val
make_deits_clues('val', 'disj')     # produces a clue file in k_deits_clue_folder
# between these need to run deits/validate_cryptics.py to actually generate the outputs
# now we have an output in the deits output folder
eval_deits_clues('val', 'disj')     # will evaluate the outputs

# can also do 'val', 'naive' and 'test' + ['naive', 'disj']

